### import

In [1]:
import pickle
import re
import os
import graphviz

f_Nodes= open('./Nodes.dbm','rb')
f_Concepts= open('./ConceptsStr.dbm','rb')
f_perepick= open('./PeresDeConcepts.dbm','rb')
f_filspick= open('./FilsDeConcepts.dbm','rb')
f_atteqpick= open('./AttEquivalents.dbm','rb')
f_objeqpick= open('./ObjEquivalents.dbm','rb')
f_AttPereInit= open('./AttPeresImmediats.dbm','rb')
f_AttPereTransitive= open('./AttPereFermTransitive.dbm','rb')
f_ListePaires= open('./ListeDesPaires.dbm','rb')
f_AttOcc= open('./OccurenceDesAttributs.dbm','rb')
f_dotDict= open('./dotDict.dbm','rb')

Nodes=pickle.load(f_Nodes)
Concepts=pickle.load(f_Concepts)
pere1=pickle.load(f_perepick)
fils= pickle.load(f_filspick)
#EquiAtt=pickle.load(f_atteqpick)
#EquiObj= pickle.load(f_objeqpick)
pereimmediat= pickle.load(f_AttPereInit)
pere= pickle.load(f_AttPereTransitive)
PairFullList= pickle.load(f_ListePaires)
Att_cpt=pickle.load(f_AttOcc)
dotDict=pickle.load(f_dotDict)

### Profiling

In [2]:
appeared_att_set=set()
for k,v in Nodes.items():
    for att in v[6]:
        appeared_att_set.add(att)
        
## subgraphs that do NOT appear in the concepts
attribute_ls=list(dotDict.keys())
no_appeared_att_set=set()
for relation in attribute_ls:
    if str(relation) not in appeared_att_set:
        no_appeared_att_set.add(relation)
#len(no_appeared_att_set) -> 2428

## subgraphs that DO appear in the concepts
is_appeared_att_set=set()
for relation in attribute_ls:
    if str(relation) in appeared_att_set:
        is_appeared_att_set.add(relation)
#len(is_appeared_att_set) -> 1278

## make clean appeared_att_dotDict
appeared_att_dotDict = dotDict.copy()
for k,v in dotDict.items():
    if k not in is_appeared_att_set:
        appeared_att_dotDict.pop(k)
#len(appeared_att_dotDict) -> 1278

In [3]:
### have the functions run first below "Local attributes with the most complex structural patterns in a concept"
## list of concepts containing single or mutliple local attribute
path_name='HierarchiesBtwAttributesPerConcepts'
single_attribute_concept_ls=[]
multi_attribute_concept_ls=[]
for file in get_files(path_name):
    if len(Nodes[file[3:-7]][2]) == 1:
        single_attribute_concept_ls.append(file[3:-7])
    else:
        multi_attribute_concept_ls.append(file[3:-7])
# len(single_attribute_concept_ls) -> 190
# len(multi_attribute_concept_ls) -> 189

## list of concepts containing none of local attributes but inherited ones
only_inherited_concepts=[]
for c in multi_attribute_concept_ls:
    if c not in concept_local_dict:
        only_inherited_concepts.append(c)
# len(only_inherited_concepts) -> 9 ['89', '92', '14', '73', '17', '199', '148', '93', '121']

NameError: name 'get_files' is not defined

### Filtering

### make set of attributes containing only single r_multi and a_joint node

In [3]:
single_multin_joint_node_ls=set()
multi_multin_joint_node_one_rel_ls=set()
for k,v in appeared_att_dotDict.items():
    r=0
    a=0
    cache_ls=[]
    ## see how many a_joint or r_multi nodes are there in a subgraph
    for rel in dotDict[k][0]:
        if re.search(r'a\_joint\d{1,3}',rel):
            cache_ls.append(re.search(r'a\_joint\d{1,3}',rel).group(0))
        if re.search(r'r_multi\d{1,3}',rel):
            cache_ls.append(re.search(r'r_multi\d{1,3}',rel).group(0))
            
    ## see how many [label="joint-a"] or RST multinuclear relations are pointing toward the a_joint or r_multi nodes
    # if single node only
    if len(cache_ls) == 1:
        # ARG
        if re.search(r'a',cache_ls[0]):
            for rel in dotDict[k][1]:
                if re.search(r'\[label="joint-a"\]',rel[2]):
                    a+=1
        # RST
        elif re.search(r'r',cache_ls[0]):
            for rel in dotDict[k][1]:
                if re.search(r'r_multi[\d{1,4}]?',rel[1]):
                    r+=1
        # add it to the list of single node 
        if r == 1 or a == 1:
            single_multin_joint_node_ls.add(k)
            
    # if multi nodes, inspect relations pointing toward them                
    elif len(cache_ls) > 1:
        # ARG
        if re.search(r'a',cache_ls[0]):
            cache_dict=dict()
            for joint_node in cache_ls:
                joint_node_digit = str(joint_node[-1])
                joint_node_search_str = 'a\_joint' + joint_node_digit
                joint_search_compile = re.compile(joint_node_search_str)
                count=0
                for link in dotDict[k][1]:
                    if re.search(joint_search_compile,link[1]) and re.search(r'\[label="joint-a"\]',link[2]):
                        count+=1
                cache_dict[joint_node]=count
                for jo,co in cache_dict.items():
                    if co == 1: 
                        multi_multin_joint_node_one_rel_ls.add(k)
        # RST
        elif re.search(r'r',cache_ls[0]):
            cache_dict=dict()
            for multi_node in cache_ls:
                multi_node_digit = str(multi_node[-1])
                multi_node_search_str = 'r\_multi' + multi_node_digit
                multi_search_compile = re.compile(multi_node_search_str)
                count=0
                cache_dict=dict()
                for link in dotDict[k][1]:
                    if re.search(multi_search_compile,link[1]):
                        count+=1
                cache_dict[multi_node]=count
                for jo,co in cache_dict.items():
                    if co == 1:
                        multi_multin_joint_node_one_rel_ls.add(k)

# len(single_multin_joint_node_ls) -> 197
# len(multi_multin_joint_node_one_rel_ls) -> 216

## see how many are there of ARG and RST in the set of relations conctaing single a_joint or r_multi node
s_a_ls=[]
s_r_ls=[]
for att in single_multin_joint_node_ls:
    if re.match(r'a\d{1,4}',att):s_a_ls.append(att)
    if re.match(r'r\d{1,4}',att):s_r_ls.append(att)
# len(s_a_ls)-> 53
# len(s_r_ls) -> 144
## see how many are there of ARG and RST in the set of relations conctaing multiple a_joint or r_multi nodes but one relation pointing toward them
m_a_ls=[]
m_r_ls=[]
for att in multi_multin_joint_node_one_rel_ls:
    if re.match(r'a\d{1,4}',att):m_a_ls.append(att)
    if re.match(r'r\d{1,4}',att):m_r_ls.append(att)
# len(m_a_ls)-> 114
# len(m_r_ls) -> 102

### make set of attributes containing only single a_rel node

In [ ]:
arel_node_ls=set()
for k,v in appeared_att_dotDict.items():
    if re.search('a',k):
    ## see how many a_joint or r_multi nodes are there in a subgraph
    for rel in dotDict[k][1]:
        if re.search(r'a\_rel\d{1,3}',rel[0]):
            rel_str=re.search(r'a\_rel\d{1,3}',rel).group(0)
            
        if re.search(r'r_multi\d{1,3}',rel):
            cache_ls.append(re.search(r'r_multi\d{1,3}',rel).group(0))
            

In [15]:
for k,v in appeared_att_dotDict.items():
    print(appeared_att_dotDict[k][1][0])
    
    break

('r_0', 'r_multi0', '[label=conjunction]')


### make set of argumentation attribute containing only a_joint node and join relation

In [5]:
joint_only_ls=set()
joint_not_only_ls=set()
for k,v in appeared_att_dotDict.items():
    intermediate_ls = list()
    for rel in dotDict[k][1]:
        intermediate_ls.append(str(rel[2]))
    if all(label == '[label="joint-a"]' for label in intermediate_ls):joint_only_ls.add(k)
    elif any(label == '[label="joint-a"]' for label in intermediate_ls):joint_not_only_ls.add(k)
#len(joint_only_ls) -> 3 (a17,a19,a311)
#len(joint_not_only_ls) -> 320

## remove attribute only with one single joint node
to_discard_ls=[]
for rel in joint_not_only_ls:
    if rel in single_multin_joint_node_ls:
        to_discard_ls.append(rel)
for to_discard in to_discard_ls:
    joint_not_only_ls.discard(to_discard)
#len(to_discard_ls) -> 53

### Local attributes with the most complex structural patterns in a concept

In [6]:
def get_files(path_name='HierarchiesBtwAttributesPerConcepts'):
    '''
    INPUT 
    path_name: str
    ---
    OUTPUT
    list_of_files: list of filenames of matched string
    '''
    directory = os.fsencode(path_name)
    list_of_files = [f.decode("utf-8") for f in os.listdir(directory) if re.match(r'Cpt\d{1,4}P-H\.dot', f.decode("utf-8"))]
    #len(list_of_files) -> 379
    return list_of_files

In [7]:
def read_files(path_name, filename):
    '''
    INPUT 
    path_name: str
    filemame: str
    ---
    OUTPUT
    digraph_tree_list: list of elements of digraph G 
    '''
    file=graphviz.Source.from_file(path_name+'/'+filename)
    f_source= file.source
    digraph_tree_list =[]
    for line in f_source.splitlines():
        digraph_tree_list.append(line)
    return digraph_tree_list

In [8]:
def get_local_attributes(digraph_tree_list):
    '''
    INPUT 
    digraph_tree_list: list of elements of digraph G 
    ---
    OUTPUT
    list_of_local_attributes: list of local attributes in strings
    '''
    list_of_local_attributes=[]
    for i in range(len(digraph_tree_list)):
        if re.search(r'fillcolor=khaki1',digraph_tree_list[i]):
            katt =re.search(r'\(?(a|r)+\d{1,4}\,?((a|r)+\d{1,4})?\)?',digraph_tree_list[i]).group(0)
            list_of_local_attributes.append(katt)
    return list_of_local_attributes

In [9]:
def get_index_of_linkage(digraph_tree_list):
    '''
    INPUT 
    digraph_tree_list: list of elements of digraph G 
    ---
    OUTPUT
    sliced digraph_tree_list: list of elements of digraph G which contain a '->' arrow as edges
    '''
    ind=[]
    a=1
    for i in range(len(digraph_tree_list)):
        if re.search(r'->',digraph_tree_list[i]):
            ind.append(a)
            a+=1
            # 1 digraph_tree_list[ind.index(1)]
            # a digraph_tree_list[ind.index(a-1)]
        else:
            ind.append(0)
    sliced_digraph_tree_list= digraph_tree_list[ind.index(1):ind.index(a-1)+1]
    return sliced_digraph_tree_list

In [10]:
def get_local_attribute_and_linkage(list_of_local_attributes, sliced_digraph_tree_list):
    '''
    INPUT 
    sliced_digraph_tree_list: list of elements of digraph G which contain a '->' arrow as edges 
    ---
    OUTPUT
    match_dict: local attributes as keys and their matched linkages as values
    '''
    match_dict=dict()
    
    ## concstruct re,compile string for search
    for local in list_of_local_attributes:
        if re.search(r'\(',local):
            local_linkage = ' -> ' + '"\\('+str(local[1:-1])+'\\)"'
        else:
            local_linkage = ' -> ' + '"'+str(local)+'"'
        # compile    
        local_linkage_compile= re.compile(local_linkage)
        ## start matching
        cache_ls=[]
        for linkage in sliced_digraph_tree_list:
            if re.search(local_linkage_compile,linkage):
                cache_ls.append(linkage)
        # build output dict {local attribute : list of matched linkage}
        match_dict[local]=cache_ls
    
    return match_dict

In [11]:
path_name='HierarchiesBtwAttributesPerConcepts'
concept_local_dict=dict()

for file in get_files(path_name):
    if len(Nodes[file[3:-7]][2]) == 1:
        continue
    digraph_tree_list= read_files(path_name,file)
    list_of_local_attributes=get_local_attributes(digraph_tree_list)
    sliced_digraph_tree_list=get_index_of_linkage(digraph_tree_list)
    match_dict=get_local_attribute_and_linkage(list_of_local_attributes, sliced_digraph_tree_list)
    for k,v in match_dict.items():
        k_ls=[]
        if len(v)==0:
            k_ls.append(k)
            concept_local_dict[file[3:-7]]= k_ls
# len(concept_local_dict) -> 180

In [ ]:
#def get_most_complex_local_attribute(match_dict):
    '''
    The most complex local attribute is not directed/pointed by any other nodes at all, thus is the
    ---
    INPUT 
    match_dict: local attributes as keys and their matched linkages as values 
    ---
    OUTPUT
    most_complex_local_attribute: string of the most complex local attribute
    '''
#    for k, v in match_dict.items():
#        if len(v)==0:
#            most_complex_local_attribute = k
#    return most_complex_local_attribute

### Matching specific relation configurations

In [ ]:
# find all relation labels
a_label_name=set()
r_label_name=set()
for k,v in appeared_att_dotDict.items():
    # ARG
    if re.search(r'a',k):
        for link in v[1]:
            if link[2] not in a_label_name:
                a_label_name.add(link[2])
    # RST
    if re.search(r'r',k):
        for link in v[1]:
            if link[2] not in r_label_name:
                r_label_name.add(link[2])   
                
# len(a_label_name) -> 6
# len(r_label_name) -> 26
a_label_name_unique_ls=list(a_label_name)
r_label_name_unique_ls=list(r_label_name)

a_name_unique_set=set()
r_name_unique_set=set()
for lab in a_label_name_unique_ls:
    a_name_unique_set.add(lab[7:-1])
for lab in r_label_name_unique_ls:
    r_name_unique_set.add(lab[7:-1])

#### Rebuttal-Undercutter

In [ ]:
# setting storing set
rebuttal_and_undercutter_set=set() 
for k,v in appeared_att_dotDict.items(): # pre-filtered dot dictionary
    # ARG
    if re.search(r'a',k):
        # for loop on every edge description in dot source file
        for rel in v[1]: 
            # if the reception node is named 'a_rel' 
            # and incoming relation is Undercutter
            if re.search(r'a_rel\d{1,3}',rel[1]) and re.search(r'und',rel[2]):
            
                # another for loop on every edge description in dot source file
                for rel in v[1]:
                    # if the projection node is also a 'a_rel'
                    # and also the projecting relation is Rebuttal
                    if re.search(r'a_rel\d{1,3}',rel[0]) and re.search(r'reb',rel[2]):
                        # when two requirements meet, add the storing set
                        rebuttal_and_undercutter_set.add(k)

# searching each RST correspondences of matched-pattern argumentation attributes
matched_pair_dict=dict()
for rel in rebuttal_and_undercutter_set:
    matched_pair_list=[]
    re_compile = re.compile(rel)
    
    # search in the list of alignment pair
    for el in PairFullList:
        if re.search(re_compile,el):
            matched_pair_list.append(el)
    matched_pair_dict[rel]=matched_pair_list

#### more specific complex configuration

In [ ]:
# rebuttal-undercutter-rebuttal
rebuttal_and_undercutter_rebuttal_set=set()
for att in rebuttal_and_undercutter_set:
    for edge in appeared_att_dotDict[att][1]:
        if re.search(r'a_rel\d{1,3}',edge[0]) and re.search(r'reb',edge[2]):
            recep_node = re.compile(str(edge[0]))
            for ed in appeared_att_dotDict[att][1]:
                if re.search(recep_node,ed[1]) and re.search(r'reb',ed[2]):
                    rebuttal_and_undercutter_rebuttal_set.add(att)
# len(rebuttal_and_undercutter_rebuttal_set) -> 59

# rebuttal-undercutter-undercutter
rebuttal_and_undercutter_undercutter_set=set()
for att in rebuttal_and_undercutter_set:
    for edge in appeared_att_dotDict[att][1]:
        if re.search(r'a_rel\d{1,3}',edge[0]) and re.search(r'reb',edge[2]):
            recep_node = re.compile(str(edge[0]))
            for ed in appeared_att_dotDict[att][1]:
                if re.search(recep_node,ed[1]) and re.search(r'und',ed[2]):
                    rebuttal_and_undercutter_undercutter_set.add(att)
# len(rebuttal_and_undercutter_undercutter_set) -> 2

# rebuttal-undercutter-support
rebuttal_and_undercutter_support_set=set()
for att in rebuttal_and_undercutter_set:
    for edge in appeared_att_dotDict[att][1]:
        if re.search(r'a_rel\d{1,3}',edge[0]) and re.search(r'reb',edge[2]):
            recep_node = re.compile(str(edge[0]))
            for ed in appeared_att_dotDict[att][1]:
                if re.search(recep_node,ed[1]) and re.search(r'sup',ed[2]):
                    rebuttal_and_undercutter_support_set.add(att)
# len(rebuttal_and_undercutter_support_set) -> 0

In [ ]:
# rebuttal-undercutter--support
rebuttal_and_undercutter_fsupport_set=set()
for att in rebuttal_and_undercutter_set:
    for edge in appeared_att_dotDict[att][1]:
        if re.search(r'a_rel\d{1,3}',edge[1]) and re.search(r'und',edge[2]):
            recep_node = re.compile(str(edge[0]))
            for ed in appeared_att_dotDict[att][1]:
                if re.search(recep_node,ed[1]) and re.search(r'sup',ed[2]):
                    rebuttal_and_undercutter_fsupport_set.add(att)
# len(rebuttal_and_undercutter_fsupport_set) -> 8

# find this configuration pair in RST
smatched_pair_dict=dict()
for rel in rebuttal_and_undercutter_fsupport_set:
    matched_pair_list=[]
    re_compile = re.compile(rel)
    
    # search in the list of alignment pair
    for el in PairFullList:
        if re.search(re_compile,el):
            matched_pair_list.append(el)
    smatched_pair_dict[rel]=matched_pair_list

#---------------------------------------------------#
# rebuttal-undercutter--rebuttal
rebuttal_and_undercutter_frebuttal_set=set()
for att in rebuttal_and_undercutter_set:
    for edge in appeared_att_dotDict[att][1]:
        if re.search(r'a_rel\d{1,3}',edge[1]) and re.search(r'und',edge[2]):
            recep_node = re.compile(str(edge[0]))
            for ed in appeared_att_dotDict[att][1]:
                if re.search(recep_node,ed[1]) and re.search(r'reb',ed[2]):
                    rebuttal_and_undercutter_frebuttal_set.add(att)
# len(rebuttal_and_undercutter_frebuttal_set) -> 0

#---------------------------------------------------#
# rebuttal-undercutter--undcutter
rebuttal_and_undercutter_fundcutter_set=set()
for att in rebuttal_and_undercutter_set:
    for edge in appeared_att_dotDict[att][1]:
        if re.search(r'a_rel\d{1,3}',edge[1]) and re.search(r'und',edge[2]):
            recep_node = re.compile(str(edge[0]))
            for ed in appeared_att_dotDict[att][1]:
                if re.search(recep_node,ed[1]) and re.search(r'und',ed[2]):
                    rebuttal_and_undercutter_fundcutter_set.add(att)
# len(rebuttal_and_undercutter_fundcutter_set) -> 0

#### Support-Undercutter

In [ ]:
## support and undercutter
support_and_undercutter_set=set()
for k,v in appeared_att_dotDict.items():
    if re.search(r'a',k):
        for rel in v[1]:
            if re.search(r'a_rel\d{1,3}',rel[1]) and re.search(r'und',rel[2]):
                for rel in v[1]:
                    if re.search(r'a_rel\d{1,3}',rel[0]) and re.search(r'sup',rel[2]):
                        support_and_undercutter_set.add(k)
# len(support_and_undercutter_set) -> 30

## remove single a_joint or JOIN relation only subgraphs
overlap1=support_and_undercutter_set.intersection(single_multin_joint_node_ls) # len(overlap1) -> 1
overlap2=support_and_undercutter_set.intersection(multi_multin_joint_node_one_rel_ls) # len(overlap2) -> 0
overlap=overlap1.union(overlap2) # len(overlap) -> 
for lap in overlap:
    if lap in support_and_undercutter_set:
        support_and_undercutter_set.remove(lap) # len(rebuttal_and_undercutter_C_set) -> 29

### Count mapping relation

In [ ]:
## Firstly search for its alignment in matched_pair_dict which defined previously while matching specific configurations

In [ ]:
## Counting RST's relaiton of an ARG alignment 
rname_unique_dict=dict.fromkeys(r_name_unique_set, 0) 

# specify the ARG attribute in between 'a62'
for pair in matched_pair_dict['a62']:
    if re.search(r'r\d{1,4}',pair):
        r_att=re.search(r'r\d{1,4}',pair).group(0)
        for edge in appeared_att_dotDict[r_att][1]:
            rname_unique_dict[edge[2][7:-1]] += 1
            
# descending order to see which relation get mapped the most
# sorted(rname_unique_dict.items(), key=lambda x: x[1], reverse=True)

In [ ]:
## Counting ARG's relaiton of a RST alignment 
aname_unique_dict=dict.fromkeys(a_name_unique_set, 0) 

# specify the RST attribute in between 'r1'
for pair in matched_pair_dict['r1']:
    if re.search(r'a\d{1,4}',pair):
        r_att=re.search(r'a\d{1,4}',pair).group(0)
        for edge in appeared_att_dotDict[r_att][1]:
            aname_unique_dict[edge[2][7:-1]] += 1
            
# descending order to see which relation get mapped the most
# sorted(aname_unique_dict.items(), key=lambda x: x[1], reverse=True)